# 5.2 Classifier SBERT

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## Read in Data

In [ ]:
import json
import numpy as np
import pickle
import gc

In [ ]:
with open ('./drive/My Drive/LAB/COMP90042 A3/data/curated/train_claims2.json') as f:
    train_claims = json.load(f)

In [ ]:
with open ('./drive/My Drive/LAB/COMP90042 A3/data/raw/dev-claims.json') as f:
    dev_claims = json.load(f)

In [ ]:
with open ('./drive/My Drive/LAB/COMP90042 A3/data/curated/test_claims2.json') as f:
    test_claims = json.load(f)

In [ ]:
with open ('./drive/My Drive/LAB/COMP90042 A3/data/raw/test-claims-unlabelled.json') as f:
    future_claims = json.load(f)

In [ ]:
with open ('./drive/My Drive/LAB/COMP90042 A3/data/raw/evidence.json') as f:
    evidence = json.load(f)

In [ ]:
import random
random.seed(19260817)

In [ ]:
evid_id_list = [evid_id for evid_id in evidence]

### Create dataset used to train retriever

In [ ]:
ENCODING = {'REFUTES': 0, 'DISPUTED': 1, 'NOT_ENOUGH_INFO': 2, 'SUPPORTS': 3}
DECODING = {ENCODING[key]:key for key in ENCODING}

In [ ]:
training_data = []

for id in train_claims:

  claim_text = train_claims[id]['claim_text']

  label = ENCODING[train_claims[id]['claim_label']] # LABELS MUST BE ENCODED!!!!!!

  n_evid = len(train_claims[id]['evidences'])
  
  for evid_id in train_claims[id]['evidences']:
    evid_text = evidence[evid_id]

    training_data.append(((claim_text, evid_text), label))

In [ ]:
dev_data = []

for id in dev_claims:

  claim_text = dev_claims[id]['claim_text']

  label = ENCODING[dev_claims[id]['claim_label']] # LABELS MUST BE ENCODED!!!!!!

  n_evid = len(dev_claims[id]['evidences'])
  
  for evid_id in dev_claims[id]['evidences']:
    evid_text = evidence[evid_id]
    
    dev_data.append(((claim_text, evid_text), label))

In [ ]:
test_data = []

for id in test_claims:

  claim_text = test_claims[id]['claim_text']

  label = ENCODING[test_claims[id]['claim_label']] # LABELS MUST BE ENCODED!!!!!!

  n_evid = len(test_claims[id]['evidences'])
  
  for evid_id in test_claims[id]['evidences']:
    evid_text = evidence[evid_id]

    test_data.append(((claim_text, evid_text), label))

In [ ]:
!pip install torch torchvision transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 95.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 27.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 105.7 MB/s eta 0:00:00


## Build model, dataloader etc

In [ ]:
from transformers import BertModel

In [ ]:
from transformers import BertTokenizer

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

In [ ]:
import torch
from torch.utils.data import Dataset
from transformers import BertTokenizer
import pandas as pd

class Dataset():

    def __init__(self, data, maxlen):

        #Store the contents of the file in a pandas dataframe
        self.data = data

        #Initialize the BERT tokenizer
        self.tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

        self.maxlen = maxlen

    def __len__(self):
        return len(self.data)

    def __getitem__(self, index):

        #Selecting the sentence and label at the specified index in the data frame
        sentence1 = self.data[index][0][0]
        sentence2 = self.data[index][0][1]
        label = self.data[index][1]

        #Preprocessing the text to be suitable for BERT
        tokens1 = tokenizer.tokenize(sentence1)
        tokens2 = tokenizer.tokenize(sentence2)
        tokens1 = ['[CLS]'] + tokens1 + ['[SEP]']
        tokens2 = ['[CLS]'] + tokens2 + ['[SEP]']
        if len(tokens1) < self.maxlen:
            tokens1 = tokens1 + ['[PAD]' for _ in range(self.maxlen - len(tokens1))] #Padding sentences
        else:
            tokens1 = tokens1[:self.maxlen-1] + ['[SEP]'] #Prunning the list to be of specified max length

        if len(tokens2) < self.maxlen:
            tokens2 = tokens2 + ['[PAD]' for _ in range(self.maxlen - len(tokens2))] #Padding sentences
        else:
            tokens2 = tokens2[:self.maxlen-1] + ['[SEP]'] #Prunning the list to be of specified max length

        tokens_ids1 = self.tokenizer.convert_tokens_to_ids(tokens1) #Obtaining the indices of the tokens in the BERT Vocabulary
        tokens_ids_tensor1 = torch.tensor(tokens_ids1) #Converting the list to a pytorch tensor

        tokens_ids2 = self.tokenizer.convert_tokens_to_ids(tokens2) #Obtaining the indices of the tokens in the BERT Vocabulary
        tokens_ids_tensor2 = torch.tensor(tokens_ids2) #Converting the list to a pytorch tensor

        #Obtaining the attention mask i.e a tensor containing 1s for no padded tokens and 0s for padded ones
        attn_mask1 = (tokens_ids_tensor1 != 0).long()
        attn_mask2 = (tokens_ids_tensor2 != 0).long()



        return tokens_ids_tensor1, attn_mask1, tokens_ids_tensor2, attn_mask2,  label

In [ ]:
from torch.utils.data import DataLoader

#Creating instances of training and development set
#maxlen sets the maximum length a sentence can have
#any sentence longer than this length is truncated to the maxlen size
train_set = Dataset(training_data, maxlen = 512)
dev_set = Dataset(dev_data, maxlen = 512)
test_set = Dataset(test_data, maxlen = 512)

#Creating intsances of training and development dataloaders
train_loader = DataLoader(train_set, batch_size = 8, shuffle = True, num_workers = 2)
dev_loader = DataLoader(dev_set, batch_size = 8, shuffle = True, num_workers = 2)
test_loader = DataLoader(test_set, batch_size = 8, shuffle = True, num_workers = 2)

In [ ]:
import torch
import torch.nn as nn
from transformers import BertModel

class nEvidClassifier(nn.Module):

    def __init__(self):
        super(nEvidClassifier, self).__init__()
        #Instantiating BERT model object 
        self.bert_layer1 = BertModel.from_pretrained('bert-base-uncased')
        self.bert_layer2 = BertModel.from_pretrained('bert-base-uncased')
        
        #Classification layer
        #input dimension is 768 because [CLS] embedding has a dimension of 768
        #output dimension is 1 because we're working with a binary classification problem
        self.cls_layer = nn.Linear(1537, 4)

        self.softmax_layer = nn.Softmax(dim=4)

    def forward(self, seq1, attn_masks1, seq2, attn_masks2):
        '''
        Inputs:
            -seq : Tensor of shape [B, T] containing token ids of sequences
            -attn_masks : Tensor of shape [B, T] containing attention masks to be used to avoid contibution of PAD tokens
        '''

        batch_size = seq1.size(0)
        #Feeding the input to BERT model to obtain contextualized representations
        claim_outputs = self.bert_layer1(seq1, attention_mask = attn_masks1, return_dict=True)
        claim_cont_reps = claim_outputs.last_hidden_state

        #Obtaining the representation of [CLS] head (the first token)
        claim_cls_reps = claim_cont_reps[:, 0]

        #Feeding the input to BERT model to obtain contextualized representations
        evid_outputs = self.bert_layer2(seq2, attention_mask = attn_masks2, return_dict=True)
        evid_cont_reps = evid_outputs.last_hidden_state

        #Obtaining the representation of [CLS] head (the first token)
        evid_cls_reps = evid_cont_reps[:, 0]

        # Concatenate the two output tensors along the last dimension (i.e., the features dimension)
        concat_output = torch.cat((claim_cls_reps, evid_cls_reps), dim=-1)

        # Calculate the Euclidean distance between the two output tensors and flatten the result
        distances = []
        for i in range(batch_size):
            distance = torch.dist(claim_cls_reps[i], evid_cls_reps[i], p=2)
            distances.append(distance)
        distances = torch.flatten(torch.stack(distances)).unsqueeze(1)

        # Concatenate the flattened distance with the concatenated output tensor
        concat_output = torch.cat((concat_output, distances), dim=-1)

        #Feeding cls_rep to the classifier layer
        logits = self.cls_layer(concat_output)

        return logits

    def get_claim_embedding(self, seq1, attn_masks1):
        #Feeding the input to BERT model to obtain contextualized representations
        claim_outputs = self.bert_layer1(seq1, attention_mask = attn_masks1, return_dict=True)
        claim_cont_reps = claim_outputs.last_hidden_state

        #Obtaining the representation of [CLS] head (the first token)
        claim_cls_reps = claim_cont_reps[:, 0]

        return claim_cls_reps
    

    def get_evid_embedding(self, seq2, attn_masks2):
        #Feeding the input to BERT model to obtain contextualized representations
        evid_outputs = self.bert_layer2(seq2, attention_mask = attn_masks2, return_dict=True)
        evid_cont_reps = evid_outputs.last_hidden_state

        #Obtaining the representation of [CLS] head (the first token)
        evid_cls_reps = evid_cont_reps[:, 0]

        return evid_cont_reps
    

    def neural_layer(self, claim_cls_reps, evid_cls_reps):

        # Concatenate the two output tensors along the last dimension (i.e., the features dimension)
        batch_size = claim_cls_reps.size(0)
        concat_output = torch.cat((claim_cls_reps, evid_cls_reps), dim=-1)
        
        # Calculate the Euclidean distance between the two output tensors and flatten the result
        distances = []
        for i in range(batch_size):
            distance = torch.dist(claim_cls_reps[i], evid_cls_reps[i], p=2)
            distances.append(distance)
        distances = torch.flatten(torch.stack(distances))
        
        # Concatenate the flattened distance with the concatenated output tensor
        concat_output = torch.cat((concat_output, distances), dim=-1)

        #Feeding cls_rep to the classifier layer
        logits = self.cls_layer(concat_output)
        

        return logits

In [ ]:
gpu = 0 #gpu ID

print("Creating the sentiment regressor, initialised with pretrained BERT-BASE parameters...")
net = nEvidClassifier()
net.cuda(gpu) #Enable gpu support for the model
print("Done creating the sentiment regressor.")

Creating the sentiment regressor, initialised with pretrained BERT-BASE parameters...


Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.seq_relationship.bias', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.seq_relationship.bias', 'cls.predictions.bias', 

Done creating the sentiment regressor.


### Setup Training

In [ ]:
import torch.nn as nn
import torch.optim as optim

criterion = nn.CrossEntropyLoss()
opti = optim.Adam(net.parameters(), lr = 2e-5)

In [ ]:
def get_accuracy_from_logits(pseudo_probs, labels):

    correct = 0
    total = 0
    for i in range(len(labels)):
        _, predicted = torch.max(pseudo_probs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    return correct/total

    # index = torch.argmax(pseudo_probs.unsqueeze(-1))
    # soft_probs = (probs > 0.5).long()
    # acc = (soft_probs.squeeze() == labels).float().mean()
    # return acc
    

def evaluate(net, criterion, dataloader, gpu):
    net.eval()

    mean_acc, mean_loss = 0, 0
    count = 0
    acc = 0

    with torch.no_grad():
        for tokens_ids_tensor1, attn_mask1, tokens_ids_tensor2, attn_mask2, label in dataloader:
            tokens_ids_tensor1, attn_mask1, tokens_ids_tensor2, attn_mask2, label = tokens_ids_tensor1.cuda(gpu), attn_mask1.cuda(gpu), tokens_ids_tensor2.cuda(gpu), attn_mask2.cuda(gpu), label.cuda(gpu)
            logits = net(tokens_ids_tensor1, attn_mask1, tokens_ids_tensor2, attn_mask2)
            _, predicted = torch.max(logits.data, 1)
            acc += (predicted == label).sum().item() / len(label)
            mean_loss += criterion(logits, label).item()
            count += 1

    return acc / count, mean_loss / count

In [ ]:
import time

def train(net, criterion, opti, train_loader, dev_loader, max_eps, gpu):

    best_acc = 0
    st = time.time()
    for ep in range(max_eps):
        
        net.train()
        for it, (tokens_ids_tensor1, attn_mask1, tokens_ids_tensor2, attn_mask2, label) in enumerate(train_loader):
            #Clear gradients
            opti.zero_grad()  
            #Converting these to cuda tensors
            tokens_ids_tensor1, attn_mask1, tokens_ids_tensor2, attn_mask2, label = tokens_ids_tensor1.cuda(gpu), attn_mask1.cuda(gpu), tokens_ids_tensor2.cuda(gpu), attn_mask2.cuda(gpu), label.cuda(gpu)

            #Obtaining the logits from the model
            output = net(tokens_ids_tensor1, attn_mask1, tokens_ids_tensor2, attn_mask2)

            #Computing loss
            loss = criterion(output, label)

            #Backpropagating the gradients
            loss.backward()

            #Optimization step
            opti.step()
              
            if it % 100 == 0:
                
                _, predicted = torch.max(output.data, 1)
                acc = (predicted == label).sum().item() / len(label)
                print("Iteration {} of epoch {} complete. Loss: {}; Accuracy: {}; Time taken (s): {}".format(it, ep, loss.item(), acc, (time.time()-st)))
                st = time.time()

        
        dev_acc, dev_loss = evaluate(net, criterion, dev_loader, gpu)
        print("Epoch {} complete! Development Accuracy: {}; Development Loss: {}".format(ep, dev_acc, dev_loss))
        if dev_acc > best_acc:
            print("Best development accuracy improved from {} to {}, saving model...".format(best_acc, dev_acc))
            best_acc = dev_acc
            torch.save(net.state_dict(), './drive/My Drive/LAB/COMP90042 A3/models/Classifiers/Classifier_SBert.dat')
            torch.save(net, './drive/My Drive/LAB/COMP90042 A3/models/Classifiers/Classifier_SBert.pt')

In [ ]:
torch.cuda.empty_cache() 

In [ ]:
num_epoch = 5

#fine-tune the model
train(net, criterion, opti, train_loader, dev_loader, num_epoch, gpu)

Iteration 0 of epoch 0 complete. Loss: 1.600810170173645; Accuracy: 0.125; Time taken (s): 1.5262272357940674
Iteration 100 of epoch 0 complete. Loss: 1.572058916091919; Accuracy: 0.0; Time taken (s): 138.86718797683716
Iteration 200 of epoch 0 complete. Loss: 1.5358103513717651; Accuracy: 0.25; Time taken (s): 139.95444679260254
Iteration 300 of epoch 0 complete. Loss: 1.3779265880584717; Accuracy: 0.25; Time taken (s): 139.89934015274048
Iteration 400 of epoch 0 complete. Loss: 1.4160341024398804; Accuracy: 0.25; Time taken (s): 140.04744601249695
Epoch 0 complete! Development Accuracy: 0.22311827956989247; Development Loss: 1.449906587600708
Best development accuracy improved from 0 to 0.22311827956989247, saving model...
Iteration 0 of epoch 1 complete. Loss: 1.4441471099853516; Accuracy: 0.125; Time taken (s): 109.96489429473877
Iteration 100 of epoch 1 complete. Loss: 1.4494788646697998; Accuracy: 0.125; Time taken (s): 140.17534637451172
Iteration 200 of epoch 1 complete. Loss: 

## Read In Model

In [ ]:
net = torch.load('./drive/My Drive/LAB/COMP90042 A3/models/temporary_models/StateClassifier_sentence.pt')

## Make Predictions (make Classifications)

In [ ]:
import torch
from torch.utils.data import Dataset
from transformers import BertTokenizer
import pandas as pd

class PredictDataset():

    def __init__(self, data, maxlen):

        #Store the contents of the file in a pandas dataframe
        self.data = data

        #Initialize the BERT tokenizer
        self.tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

        self.maxlen = maxlen

    def __len__(self):
        return len(self.data)

    def __getitem__(self, index):

        #Selecting the sentence and label at the specified index in the data frame
        sentence1 = self.data[index][0][0]
        sentence2 = self.data[index][0][1]

        #Preprocessing the text to be suitable for BERT
        tokens1 = tokenizer.tokenize(sentence1)
        tokens2 = tokenizer.tokenize(sentence2)
        tokens1 = ['[CLS]'] + tokens1 + ['[SEP]']
        tokens2 = ['[CLS]'] + tokens2 + ['[SEP]']
        if len(tokens1) < self.maxlen:
            tokens1 = tokens1 + ['[PAD]' for _ in range(self.maxlen - len(tokens1))] #Padding sentences
        else:
            tokens1 = tokens1[:self.maxlen-1] + ['[SEP]'] #Prunning the list to be of specified max length

        if len(tokens2) < self.maxlen:
            tokens2 = tokens2 + ['[PAD]' for _ in range(self.maxlen - len(tokens2))] #Padding sentences
        else:
            tokens2 = tokens2[:self.maxlen-1] + ['[SEP]'] #Prunning the list to be of specified max length

        tokens_ids1 = self.tokenizer.convert_tokens_to_ids(tokens1) #Obtaining the indices of the tokens in the BERT Vocabulary
        tokens_ids_tensor1 = torch.tensor(tokens_ids1) #Converting the list to a pytorch tensor

        tokens_ids2 = self.tokenizer.convert_tokens_to_ids(tokens2) #Obtaining the indices of the tokens in the BERT Vocabulary
        tokens_ids_tensor2 = torch.tensor(tokens_ids2) #Converting the list to a pytorch tensor

        #Obtaining the attention mask i.e a tensor containing 1s for no padded tokens and 0s for padded ones
        attn_mask1 = (tokens_ids_tensor1 != 0).long()
        attn_mask2 = (tokens_ids_tensor2 != 0).long()



        return tokens_ids_tensor1, attn_mask1, tokens_ids_tensor2, attn_mask2

In [ ]:

def get_classif(claims, file_name, SIZE):


  try:
    with open(f'./drive/My Drive/LAB/COMP90042 A3/predictions/Retrievals/{file_name}.pickle', 'rb') as f:
      predictions = pickle.load(f)
  except:
      predictions = {}


  i = 0
  for id in claims:
    if id in predictions:
      print('pass:', id)
      continue

    print(id)

    claim_text = claims[id]['claim_text']
    
    data_for_predict = list()

    for evid_id in claims[id]['evidences']:
      evid_text = evidence[evid_id]
      
      data_for_predict.append((claim_text, evid_text))

    set_for_predict = PredictDataset(data_for_predict, maxlen = 512)

    predict_loader = DataLoader(set_for_predict, batch_size = SIZE, num_workers = 2)
    
    predicted_logit = list()
    with torch.no_grad():
      for it, (seq, attn_masks, seg_ids_tensor) in enumerate(predict_loader):
        
        torch.cuda.empty_cache()
        gc.collect()
        seq, attn_masks, seg_ids_tensor = seq.cuda(gpu), attn_masks.cuda(gpu), seg_ids_tensor.cuda(gpu)

        logits = net(seq, attn_masks, seg_ids_tensor).tolist()
      
      predictions[id] = logits

      with open(f'./drive/My Drive/LAB/COMP90042 A3/predictions/Retrievals/{file_name}.pickle', 'wb') as f:
          pickle.dump(predictions, f)

claim-752
claim-375
claim-1266
claim-871
claim-2164
claim-1607
claim-761
claim-1718
claim-1273
claim-1786
claim-2796
claim-2580
claim-1219
claim-75
claim-2813
claim-2335
claim-161
claim-2243
claim-1256
claim-506
claim-369
claim-2184
claim-1057
claim-104
claim-1975
claim-139
claim-2062
claim-1160
claim-2679
claim-2662
claim-1490
claim-2768
claim-2168
claim-785
claim-2426
claim-1292
claim-993
claim-2593
claim-1567
claim-1834
claim-856
claim-540
claim-757
claim-1407
claim-3070
claim-1745
claim-1515
claim-1519
claim-3069
claim-677
claim-765
claim-2275
claim-1113
claim-2611
claim-2060
claim-2326
claim-1087
claim-2867
claim-2300
claim-2250
claim-2429
claim-3051
claim-1549
claim-261
claim-2230
claim-2579
claim-1416
claim-2497
claim-811
claim-1896
claim-2819
claim-2643
claim-1775
claim-316
claim-896
claim-331
claim-2574
claim-342
claim-2034
claim-578
claim-976
claim-1097
claim-609
claim-173
claim-1222
claim-2441
claim-756
claim-2577
claim-2890
claim-2478
claim-2399
claim-3091
claim-141
claim-1

Classifications from Ground Truth

In [ ]:
get_retrievals(dev_claims, 'Classification_BERT_dev', SIZE=32)

In [ ]:
get_retrievals(test_claims, 'Classification_BERT_test', SIZE=32)

In [ ]:
with open('./drive/My Drive/LAB/COMP90042 A3/models/predictions/Retrievals/Retrieval_BERT_redued_dev.pickle', 'rb') as f:
    dev_predicted_evidence = pickle.load(f)

dev_claims_pred = {}

for claim in dev_predicted_evidence:
  evid_list = [x[0] for x in dev_predicted_evidence[claim]['evidences']]

  dev_claims_pred[claim] = evid_list[:4]

In [ ]:
with open('./drive/My Drive/LAB/COMP90042 A3/models/predictions/Retrievals/Retrieval_BERT_redued_test.pickle', 'rb') as f:
    test_predicted_evidence = pickle.load(f)

test_claims_pred = {}

for claim in test_predicted_evidence:
  evid_list = [x[0] for x in test_predicted_evidence[claim]['evidences']]

  test_claims_pred[claim] = evid_list[:4]

In [ ]:
with open('./drive/My Drive/LAB/COMP90042 A3/models/predictions/Retrievals/Retrieval_BERT_redued_future.pickle', 'rb') as f:
    future_predicted_evidence = pickle.load(f)

future_claims_pred = {}

for claim in future_predicted_evidence:
  evid_list = [x[0] for x in future_predicted_evidence[claim]['evidences']]

  future_claims_pred[claim] = evid_list[:4]

Classifications from Predicted Evidence

In [ ]:
get_retrievals(future_claims_dev, 'Classification_BERT_devp-bert', SIZE=32)

In [ ]:
get_retrievals(future_claims_test, 'Classification_BERT_testp-bert', SIZE=32)

In [ ]:
get_retrievals(future_claims_pred, 'Classification_BERT_futurep-bert', SIZE=32)